In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
import pickle
from sklearn.neural_network import MLPClassifier
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,Flatten
from keras.optimizers import SGD

C:\Users\master\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\Users\master\Anaconda3\lib\site-packages\dask\dataframe\utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\master\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
C:\Users\master\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


In [2]:
df = pd.read_csv('../submission files/test.csv')
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
df ['Age'] = df['Age'].fillna(df['Age'].mean())
df['Fare'] = df['Fare'].fillna(df['Fare'].mean())
df.isnull().sum()
labels = ['child','adult','old']
df['age_group'] = pd.cut(df['Age'], bins=3, labels=labels)
df['age_group']= df['age_group'].fillna('adult')
labelsfare = ['cheap', 'normal', 'expensive']
df['Fare_group'] = pd.cut(df['Fare'], bins=3, labels=labelsfare)
df['Fare_group']= df['Fare_group'].fillna('cheap')
print(df.isnull().sum())
df

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
age_group        0
Fare_group       0
dtype: int64


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,age_group,Fare_group
0,892,3,"Kelly, Mr. James",male,34.50000,0,0,330911,7.8292,NaN,Q,adult,cheap
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.00000,1,0,363272,7.0000,NaN,S,adult,cheap
2,894,2,"Myles, Mr. Thomas Francis",male,62.00000,0,0,240276,9.6875,NaN,Q,old,cheap
3,895,3,"Wirz, Mr. Albert",male,27.00000,0,0,315154,8.6625,NaN,S,adult,cheap
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.00000,1,1,3101298,12.2875,NaN,S,child,cheap
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,30.27259,0,0,A.5. 3236,8.0500,NaN,S,adult,cheap
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.00000,0,0,PC 17758,108.9000,C105,C,adult,cheap
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.50000,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,adult,cheap
416,1308,3,"Ware, Mr. Frederick",male,30.27259,0,0,359309,8.0500,NaN,S,adult,cheap


In [5]:
df.drop(columns=['PassengerId', 'Name', 'Cabin', 'Embarked', 'Ticket','Fare', 'Age'], inplace=True)

In [ ]:
df

In [5]:
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
df.loc[df['FamilySize'] == 1, 'IsAlone'] = 1
df['IsAlone']= df['IsAlone'].fillna(0)
df.drop(columns=['SibSp', 'Parch', ], inplace=True)
df

,Pclass,Sex,age_group,Fare_group,FamilySize,IsAlone
0,3,male,adult,cheap,1,1.0
1,3,female,adult,cheap,2,0.0
2,2,male,old,cheap,1,1.0
3,3,male,adult,cheap,1,1.0
4,3,female,child,cheap,3,0.0
...,...,...,...,...,...,...
413,3,male,adult,cheap,1,1.0
414,1,female,adult,cheap,1,1.0
415,3,male,adult,cheap,1,1.0
416,3,male,adult,cheap,1,1.0


In [6]:
X = df.iloc[:,:].values
X

array([[3, 'male', 0, 0, 'adult', 'cheap'],
       [3, 'female', 1, 0, 'adult', 'cheap'],
       [2, 'male', 0, 0, 'old', 'cheap'],
       ...,
       [3, 'male', 0, 0, 'adult', 'cheap'],
       [3, 'male', 0, 0, 'adult', 'cheap'],
       [3, 'male', 1, 1, 'adult', 'cheap']], dtype=object)

In [7]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1,4,-1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[0.0 1.0 1.0 ... 3 0 0]
 [1.0 0.0 1.0 ... 3 1 0]
 [0.0 1.0 0.0 ... 2 0 0]
 ...
 [0.0 1.0 1.0 ... 3 0 0]
 [0.0 1.0 1.0 ... 3 0 0]
 [0.0 1.0 1.0 ... 3 1 1]]


In [8]:
sc = StandardScaler()
X[:, 2:] = sc.fit_transform(X[:, 2:])
X

array([[0.0, 1.0, 0.8403109239915862, ..., 0.8734819050636105,
        -0.4994700165357186, -0.4002477016385638],
       [1.0, 0.0, 0.8403109239915862, ..., 0.8734819050636105,
        0.6169923733676523, -0.4002477016385638],
       [0.0, 1.0, -1.1900357016065817, ..., -0.3158191904301653,
        -0.4994700165357186, -0.4002477016385638],
       ...,
       [0.0, 1.0, 0.8403109239915862, ..., 0.8734819050636105,
        -0.4994700165357186, -0.4002477016385638],
       [0.0, 1.0, 0.8403109239915862, ..., 0.8734819050636105,
        -0.4994700165357186, -0.4002477016385638],
       [0.0, 1.0, 0.8403109239915862, ..., 0.8734819050636105,
        0.6169923733676523, 0.6198958305865561]], dtype=object)

In [56]:
filename = '../titanic600.sav'
loaded_modelmlp = pickle.load(open(filename, 'rb'))

In [57]:
predictions= loaded_modelmlp.predict(X).tolist()
print(predictions)

[[0.1854683756828308], [0.8036348819732666], [0.1413656771183014], [0.1854683756828308], [0.8144630193710327], [0.16794468462467194], [0.18297068774700165], [0.1703503131866455], [0.6316170692443848], [0.1794372797012329], [0.1854683756828308], [0.44987937808036804], [0.8309708833694458], [0.17921915650367737], [0.8177700042724609], [0.7621287107467651], [0.19809643924236298], [0.16794468462467194], [0.8036348819732666], [0.18297068774700165], [0.15588420629501343], [0.45179125666618347], [0.8341443538665771], [0.37014442682266235], [0.8338358402252197], [0.149870365858078], [0.8344404697418213], [0.16794468462467194], [0.44987937808036804], [0.15146243572235107], [0.1632811725139618], [0.16367670893669128], [0.2037244737148285], [0.2037244737148285], [0.3180578351020813], [0.16794468462467194], [0.18297068774700165], [0.6316170692443848], [0.16794468462467194], [0.1854683756828308], [0.1824374496936798], [0.44987937808036804], [0.1854683756828308], [0.8228616714477539], [0.81777000427

In [26]:
filename = '../MLPClassifier30854.sav'
loaded_modelmlp = pickle.load(open(filename, 'rb'))

In [58]:
se = pd.Series(predictions)
df['check'] = se
df['check'] = df['check'].str.get(0)
print(se)
finaldf = []
for val in df.check:
    if val >= 0.5:
        finaldf.append(1)
    else:
        finaldf.append(0)

0       [0.1854683756828308]
1       [0.8036348819732666]
2       [0.1413656771183014]
3       [0.1854683756828308]
4       [0.8144630193710327]
               ...          
413     [0.1854683756828308]
414     [0.8341443538665771]
415     [0.1854683756828308]
416     [0.1854683756828308]
417    [0.14273512363433838]
Length: 418, dtype: object


In [59]:
sub['Survived'] = finaldf
sub
sub.to_csv('../submission files/readymodelker600.csv', index=False)

In [ ]:
filename = '../models/RandomForestClassifier_model2079.sav'
loaded_modelrfc = pickle.load(open(filename, 'rb'))

In [27]:
y_predrf = loaded_modelmlp.predict(X)

In [13]:
sub = pd.read_csv('../submission files/gender_submission.csv', delimiter=',')
sub.drop(columns=['Survived'], inplace=True)

In [29]:
sub['Survived'] = y_predrf
sub
sub.to_csv('../submission files/readymlp30854.csv', index=False)

In [ ]:
filename = '../models/RandomForestClassifier_model2079.sav'
loaded_modelxgb = pickle.load(open(filename, 'rb'))

In [ ]:
y_predrf = loaded_modelxgb.predict(X)

In [ ]:
sub = pd.read_csv('../submission files/gender_submission.csv', delimiter=',')
sub.drop(columns=['Survived'], inplace=True)

In [ ]:
sub['Survived'] = y_predrf
sub
sub.to_csv('../submission files/readyrf2078.csv', index=False)

#  Decision Tree 78

In [ ]:
filename = '../models/XGBClassifier_model2078.sav'
loaded_modeldt = pickle.load(open(filename, 'rb'))


In [ ]:
y_preddt = loaded_modeldt.predict(X)
y_preddt

In [ ]:
sub = pd.read_csv('../submission files/gender_submission.csv', delimiter=',')
sub

In [ ]:
sub.drop(columns=['Survived'], inplace=True)

In [ ]:
sub['Survived'] = y_preddt

In [ ]:
sub
sub.to_csv('../submission files/readyxgb2078.csv', index=False)

# KNN

In [ ]:
filename = '../KNN_model.sav'
loaded_modelknn = pickle.load(open(filename, 'rb'))

In [ ]:
y_predknn = loaded_modelknn.predict(X)
y_predknn

# LOGISTIC REGRESSION PREDICTION 76%

In [ ]:
filename = 'LogisticRegression_model.sav'
loaded_modelreg = pickle.load(open(filename, 'rb'))
X

In [ ]:
y_predreg = loaded_modelreg.predict(X)
y_predreg

In [ ]:
sub = pd.read_csv('../submission files/gender_submission.csv', delimiter=',')
sub

In [ ]:
sub.drop(columns=['Survived'], inplace=True)


In [ ]:
sub['Survived'] = y_predknn

In [ ]:
sub
sub.to_csv('../submission files/readyknn.csv', index=False)

In [ ]:
filename = 'XGBClassifier_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))
X

In [ ]:
xgb  = loaded_model.predict(X)
xgb

In [ ]:
filename = 'DecisionTree_model.sav'
load_model = pickle.load(open(filename, 'rb'))


In [ ]:
dstpredict = load_model.predict(X)

In [ ]:
sub = pd.read_csv('gender_submission.csv', delimiter=',')
sub.drop(columns=['Survived'], inplace=True)

In [ ]:
sub['Survived'] = xgb

In [ ]:
sub.to_csv('xgbready.csv', index=False)

In [ ]:
filename = 'RandomForest_model.sav'
load_randomf = pickle.load(open(filename, 'rb'))
rf = load_randomf.predict(X)
rf

In [ ]:
filename = '../XGBClassifier80_model.sav'
load_randomxg80 = pickle.load(open(filename, 'rb'))
y_predxgb80 = load_randomxg80.predict(X)

In [ ]:
sub = pd.read_csv('../submission files/gender_submission.csv', delimiter=',')
sub.drop(columns=['Survived'], inplace=True)
sub['Survived'] = y_predxgb80
sub.to_csv('../submission files/readyxgb80.csv', index=False)